In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import lightkurve as lk
from modules import *
from astropy.table import Table

plt.style.use('seaborn-v0_8-darkgrid')

TIC = 'TIC 219107776'

In [ ]:
lcs = get_lightcurves(TIC, author = 'SPOC', cadence='short')

In [ ]:
lc = pd.DataFrame({'time': lcs[1].time.btjd, 'flux': np.array(lcs[1].flux, dtype='f')})
lc.info()
lc.dropna(inplace=True)
lc.info()

In [ ]:
def straight_lines_t(lightcurve : lk.lightcurve.LightCurve, cadence_magnifier : int = 4) -> lk.lightcurve.LightCurve:
    '''
    Takes in a lightcurve and fills the gaps with a straight line
    ANDDDDDD smoothens the lightcurve with a spline interpolation with a factor of `cadence_magnifier`.

    Returns a lightcurve
    '''

    #BASICS
    flux = np.array(lightcurve.flux)
    time = np.array(lightcurve.time.jd)
    cadence_in_days = ((np.median(np.diff(time[:100])) * 86400).round())/86400
    lc = pd.DataFrame({'time': time, 'flux': flux})

    #PEAKS
    peaks, _ = signal.find_peaks(np.diff(time), height = cadence_in_days * 10)
    print("The gaps are at time(s):", peaks)

    #Filling the Gaps
    for i in peaks:
        t = (time[i], time[i+1])
        f = (flux[i], flux[i+1])
        df = pd.DataFrame({'time': t, 'flux': f})
        df.set_index('time', inplace = True)
        df = df.reindex(np.arange(time[i], time[i+1], cadence_in_days))
        df['flux'] = df['flux'].interpolate('linear')
        df.reset_index(inplace = True)
        df.rename(columns={'index':'time'}, inplace=True)
        lc = pd.concat([df[1:-1], lc]).sort_values('time')

    time_final = np.array(lc['time'])
    flux_final = np.array(lc['flux'])
    time_smooth = np.linspace(time_final.min(), time_final.max(), len(time_final) * cadence_magnifier)
    print(np.isnan(flux_final).any())
    print(np.isnan(time_final).any())
    print(np.isnan(time_smooth).any())
    print(np.isinf(flux_final).any())
    print(np.isinf(time_final).any())
    print(np.isinf(time_smooth).any())
    flux_smooth = spline(time_final, flux_final, k = 3)(time_smooth)

    # lightcurve.flux, lightcurve.time = flux_smooth, time_smooth

    disposable_lightcurve = lk.LightCurve(time = time_smooth, flux = flux_smooth)
    disposable_lightcurve.time.format = 'btjd' 

    return disposable_lightcurve

In [ ]:
new_lcs = []

for lc in lcs:
    new_lc = straight_lines_t(lc)
    new_lcs.append(new_lc)
    new_lc.plot()


In [ ]:
df = pd.DataFrame({'time':np.array(lcs[0].time.btjd), 'flux' : np.array(lcs[0].flux)})
df.dropna(inplace=True)
df.sort_values(by='time', inplace=True)

print(df.info())

df.plot(x='time', y='flux', lw=0.5, title=TIC)

In [ ]:
df